In [26]:
import os
import numpy as np
import tensorflow as tf
import keras
import nengo_dl
from tensorflow.python.keras import Input, Model
import nengo
from tensorflow.python.keras.callbacks import EarlyStopping
from tensorflow.python.keras.layers import Conv2D, Dropout, AveragePooling2D, Flatten, Dense, BatchNormalization, Conv3D
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from keras import backend as K
import pandas as pd
from sklearn import metrics
import bnci_utils as utils

In [27]:
num_participants = 18

# List of files with samples from each participant
dataset_path = os.path.join('dataset_result')
files = [os.path.join(dataset_path, 'P{:02d}.npz'.format(i+1)) for i in range(num_participants)] # P01 - P18 files

In [28]:
# set seed to produce a consistent result
seed = 2
np.random.seed(seed)
tf.random.set_seed(seed)

In [29]:
def run_individual(file, particip_num, model, params_save_path, test_size=0.25, epochs=30, scale_firing_rates=1000,
                   synapse=0.01, timesteps=50):
    print(f'Running ANN and SNN for file: {file}')

    dataset = np.load(file) # load numpy file containing the preprocessed data for specific participant
    features, labels = dataset['features'], dataset['labels'] # get features and labels from the numpy file

    # Transform numpy arrays to be usable with Nengo
    features, labels = utils.reshape_dataset(features, labels)

    # Split the data into 75% training and 25% testing
    x_train, x_test, y_train, y_test = train_test_split(features, labels, test_size=test_size, random_state=seed,
                                                        shuffle=True)
    print('X (train) shape:', x_train.shape, 'Y (train) shape:', y_train.shape)
    print('X (test) shape:', x_test.shape, 'Y (test) shape:', y_test.shape)

    ann_stats = utils.run_ann(model, (x_train, y_train), (x_test, y_test), params_save_path, particip_num, num_epochs=epochs,
                              optimizer=keras.optimizers.Adam(), loss=keras.losses.BinaryCrossentropy(), batch_size=1,
                              callbacks=[EarlyStopping(monitor='loss', patience=8, verbose=1, restore_best_weights=True)])
    snn_stats = utils.run_snn(model, x_test,  y_test, params_save_path, particip_num, batch_size=1,
                      timesteps=timesteps, synapse=synapse, scale_firing_rates=scale_firing_rates)

    return ann_stats, snn_stats

In [30]:
ann, snn = [], []
participant_no = 1

params_save_dir = 'cnn_individuals_nengo_params'
os.makedirs(params_save_dir, exist_ok=True)

for file in files:
    file_name = 'P{:02d}'.format(participant_no)
    model = utils.cnn_model(seed=seed)
    params_save_path = os.path.join(params_save_dir, file_name)

    ann_stats, snn_stats = run_individual(file, participant_no, model, params_save_path)

    ann.append(ann_stats) # append statistics to the list
    snn.append(snn_stats)

    participant_no += 1 # increase participant number

    # Delete model and clear session to prevent memory leaks
    K.clear_session()
    del model

Running ANN and SNN for file: dataset_result\P01.npz
X (train) shape: (90, 1, 5040) Y (train) shape: (90, 1, 2)
X (test) shape: (30, 1, 5040) Y (test) shape: (30, 1, 2)
Build finished in 0:00:01                                                      
Optimization finished in 0:00:00                                               
Construction finished in 0:00:00                                               
Epoch 1/30
90/90 [==============================] - 4s 11ms/step - loss: 1.4050 - probe_loss: 1.4050 - probe_accuracy: 0.5437 

c:\dev\anaconda\envs\spiking-nn\lib\site-packages\nengo_dl\converter.py:326: UserWarning: Layer type Dropout does not have a registered converter. Falling back to TensorNode.
  warnings.warn(
c:\dev\anaconda\envs\spiking-nn\lib\site-packages\nengo_dl\converter.py:588: UserWarning: Activation type <function softmax_v2 at 0x000001BE3FF90A60> does not have a native Nengo equivalent; falling back to a TensorNode
  warnings.warn(
c:\dev\anaconda\envs\spiking-nn\lib\site-packages\nengo_dl\simulator.py:1765: UserWarning: Number of elements (1) in ['ndarray'] does not match number of Nodes (4); consider using an explicit input dictionary in this case, so that the assignment of data to objects is unambiguous.
  warnings.warn(
c:\dev\anaconda\envs\spiking-nn\lib\site-packages\nengo_dl\converter.py:326: UserWarning: Layer type Dropout does not have a registered converter. Falling back to TensorNode.
  warnings.warn(
c:\dev\anaconda\envs\spiking-nn\lib\site-packages\nengo_dl\converter.py:588: User

In [31]:
df = utils.create_data_df(ann, snn, num_participants)

# Rename iterations to participants
df.rename(columns={'iterations': 'participant'})

df

,iterations,ann_accuracy,ann_precision,ann_recall,ann_f1,snn_accuracy,snn_precision,snn_recall,snn_f1
0,1,0.600000,0.625000,0.833333,0.714286,0.566667,0.619048,0.722222,0.666667
1,2,0.500000,0.483871,0.750000,0.588235,0.452381,0.448276,0.650000,0.530612
2,3,0.500000,0.000000,0.000000,0.000000,0.500000,0.000000,0.000000,0.000000
3,4,0.523810,0.000000,0.000000,0.000000,0.523810,0.000000,0.000000,0.000000
4,5,0.452381,0.444444,0.600000,0.510638,0.380952,0.392857,0.550000,0.458333
5,6,0.547619,0.547619,1.000000,0.707692,0.547619,0.547619,1.000000,0.707692
6,7,0.500000,0.500000,1.000000,0.666667,0.500000,0.500000,1.000000,0.666667
7,8,0.476190,0.476190,1.000000,0.645161,0.404762,0.435897,0.850000,0.576271
8,9,0.547619,0.555556,0.476190,0.512821,0.523810,0.521739,0.571429,0.545455
9,10,0.500000,0.500000,1.000000,0.666667,0.500000,0.500000,1.000000,0.666667


In [32]:
data_output_folder = 'individuals_output' # output path for data from each iteration
os.makedirs(data_output_folder, exist_ok=True)
df.to_excel(os.path.join(data_output_folder, 'cnn_individuals.xlsx'))

'Statistics for iterations successfully saved.'

'Statistics for iterations successfully saved.'

In [33]:
df_stats = utils.create_stats_df(df)
df_stats

,models,average_accuracy,max_accuracy,accuracy_std,average_precision,max_precision,average_recall,max_recall,average_f1,max_f1
0,ann,0.508201,0.600000,0.038954,0.394937,0.625000,0.674206,1.0,0.489831,0.714286
1,snn,0.495767,0.566667,0.049430,0.386969,0.619048,0.640785,1.0,0.467934,0.707692


In [34]:
# Save the statistics
df_stats.to_excel(os.path.join(data_output_folder, 'cnn_individuals_stats.xlsx'))

'File with statistics successfully saved.'

'File with statistics successfully saved.'

In [35]:
# Print confusion matrices
utils.print_confusion_matrices(ann, snn)


Confusion matrices for the ANN:
[[ 3  9]
 [ 3 15]] 

[[ 6 16]
 [ 5 15]] 

[[21  0]
 [21  0]] 

[[22  0]
 [20  0]] 

[[ 7 15]
 [ 8 12]] 

[[ 0 19]
 [ 0 23]] 

[[ 0 21]
 [ 0 21]] 

[[ 0 22]
 [ 0 20]] 

[[13  8]
 [11 10]] 

[[ 0 21]
 [ 0 21]] 

[[22  0]
 [20  0]] 

[[ 0 20]
 [ 0 22]] 

[[ 0 23]
 [ 0 19]] 

[[ 0 20]
 [ 0 22]] 

[[10 11]
 [11 10]] 

[[ 0 19]
 [ 0 23]] 

[[21  0]
 [21  0]] 

[[ 0 23]
 [ 0 19]] 

Confusion matrices for the SNN
[[ 4  8]
 [ 5 13]] 

[[ 6 16]
 [ 7 13]] 

[[21  0]
 [21  0]] 

[[22  0]
 [20  0]] 

[[ 5 17]
 [ 9 11]] 

[[ 0 19]
 [ 0 23]] 

[[ 0 21]
 [ 0 21]] 

[[ 0 22]
 [ 3 17]] 

[[10 11]
 [ 9 12]] 

[[ 0 21]
 [ 0 21]] 

[[22  0]
 [20  0]] 

[[ 0 20]
 [ 0 22]] 

[[ 0 23]
 [ 0 19]] 

[[ 0 20]
 [ 0 22]] 

[[17  4]
 [17  4]] 

[[ 0 19]
 [ 0 23]] 

[[21  0]
 [21  0]] 

[[ 0 23]
 [ 0 19]] 

